In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split,train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,mean_squared_error, mean_absolute_error, r2_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [ ]:
rawdata = pd.read_csv('s3://sagemaker-studio-905418013525-nvxe84zgs6/Labrado/rl8/rl8.csv')
# Convert the data types of each column
rl8 = rawdata
rl8['subject_id'] = rawdata['subject_id'].astype('Int64')
rl8['hadm_id'] = rawdata['hadm_id'].astype('Int64')
rl8['itemid'] = rawdata['itemid'].astype('Int64')
rl8['charttime'] = pd.to_datetime(rawdata['charttime'])
rl8['chartyear'] = rawdata['chartyear'].astype('Int64')

In [ ]:
print(rl8.shape)
rl8.dropna(how='any', inplace=True)
print(rl8.shape)
mapping = {50882: 'Bic', 50912: 'Crt', 50971: 'Pot', 50983: 'Sod', 51006: 'Ure', 51222: 'Hgb', 51265: 'Plt', 51301: 'Wbc'}
rl8['itemname'] = rl8['itemid'].map(mapping)
rl_sorted = rl8.sort_values(by=['hadm_id', 'chartyear'])
rl_sorted['min_timestamps'] = rl_sorted.groupby('hadm_id')['charttime'].transform('min')
rl_sorted['relative_time'] = ((rl_sorted['charttime'] - rl_sorted['min_timestamps']).dt.total_seconds() // 3600).astype(int)
rl_sorted = rl_sorted.reset_index(drop=True)
hadm_id_indices = rl_sorted.groupby('hadm_id').apply(lambda group: {'start_index': group.index.min(), 'end_index': group.index.max()+1}).to_dict()

In [1]:
LabName = 'Crt'
selected_rl = rl_sorted[rl_sorted['itemname']==LabName][['hadm_id','chartyear','itemname','valuenum','relative_time']]
for inx, row in selected_rl.iterrows():
  hadm_id = row['hadm_id']
  start_index = hadm_id_indices.get(hadm_id)['start_index']
  # Get the end index for selection
  end_index = hadm_id_indices.get(hadm_id)['end_index']
  filtered_df = pd.concat([rl_sorted.iloc[start_index:inx], rl_sorted.iloc[inx + 1:end_index]])
  filtered_df['abs_time_diff'] = abs(filtered_df['relative_time']-row['relative_time'])
  idx_min_abs_time_diff = filtered_df.groupby('itemname')['abs_time_diff'].idxmin()
  pred_df = filtered_df.loc[idx_min_abs_time_diff, ['itemname', 'valuenum','abs_time_diff']]
  for _,prow in pred_df.iterrows():
    selected_rl.at[inx,prow['itemname']] = prow['valuenum']
    selected_rl.at[inx,f"{prow['itemname']}_timedif"] = prow['abs_time_diff']
selected_rl.to_csv(f"s3://sagemaker-studio-905418013525-nvxe84zgs6/Labrado/rl8/rl8_{LabName}.csv", index=False)

NameError: name 'rl_sorted' is not defined